In [1]:
import requests
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
import osm2geojson  # pip install osm2geojson

# Bounding box for Nantes
north, south = 47.298354021640314, 47.17104668835133
west, east = -1.6681752121847566, -1.4480483659767052

# Overpass query for buildings (ways + relations)
overpass_url = "https://overpass-api.de/api/interpreter"
query = f"""
[out:json][timeout:180];
(
  way["building"]({south},{west},{north},{east});
  relation["building"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

# Request OSM data
print("⏳ Downloading OSM data for Nantes...")
response = requests.get(overpass_url, params={"data": query})
data = response.json()

# Convert to GeoJSON format
print("🔄 Converting to GeoJSON...")
geojson = osm2geojson.json2geojson(data)

# Load into GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson["features"])
gdf.set_crs("EPSG:4326", inplace=True)

# Fix any invalid geometries
print("🧼 Cleaning invalid geometries...")
gdf["geometry"] = gdf["geometry"].buffer(0)

# Merge all touching/overlapping geometries
print("🔗 Merging geometries...")
merged = unary_union(gdf["geometry"])

# Wrap into GeoDataFrame
if isinstance(merged, MultiPolygon):
    merged_gdf = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=gdf.crs)
else:
    merged_gdf = gpd.GeoDataFrame(geometry=[merged], crs=gdf.crs)

# Save result
output_file = "nantes_buildings_merged.gpkg"
merged_gdf.to_file(output_file, driver="GPKG")

print(f"✅ Done! Merged {len(gdf)} buildings into {len(merged_gdf)} blocks.")
print(f"📦 Output saved to: {output_file}")


⏳ Downloading OSM data for Nantes...
🔄 Converting to GeoJSON...
🧼 Cleaning invalid geometries...
🔗 Merging geometries...
✅ Done! Merged 197516 buildings into 95466 blocks.
📦 Output saved to: nantes_buildings_merged.gpkg
